In [13]:
import argparse
from PIL import Image
import cv2
import numpy as np
import os
import pandas as pd
import datetime
import tensorflow as tf
import sys
sys.path.append("C:/Users/AALY/object_detection/models/research/slim")
sys.path.append("C:/Users/AALY/object_detection/models/research/object_detection")
from object_detection.utils import visualization_utils as vis_util
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util


def run_inference_for_single_image(image, sess, tensor_dict):
    if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(detection_masks_reframed, 0)
    image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

    # Run inference
    output_dict = sess.run(tensor_dict, feed_dict={image_tensor: np.expand_dims(image, 0)})

    # all outputs are float32 numpy arrays, so convert types as appropriate
    output_dict['num_detections'] = int(output_dict['num_detections'][0])
    output_dict['detection_classes'] = output_dict[
        'detection_classes'][0].astype(np.uint8)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]
    if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict


def load_model(PATH_TO_FROZEN_GRAPH: str, PATH_TO_LABELS: str):
    """
    Load model from frozen inference graph
    """
    # Load frozen inference graph into memory
    graph = tf.Graph()
    with graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')

    # Loading labelmap
    category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
    return graph, category_index


def run_surveillance(model_path, labelmap_path, show_video_steam, label_to_look_for, output_directory, threshold, label_to_look_for2):
    # Load model
    graph, category_index = load_model(model_path, labelmap_path)
    # Initialize Video Capture
    cap = cv2.VideoCapture("C:/Users/AALY/object_detection/testZarbEAzb.mp4")
    # Create output directory if not already created
    os.makedirs(output_directory, exist_ok=True)
    os.makedirs(output_directory+'/images', exist_ok=True)
    showcrime=0
    showuniform=0
    police_arrived = False 
    ToWriteOnlabel = "NOT A CRIME SCENE"
    if os.path.exists(output_directory+'/results.csv'):
        df = pd.read_csv(output_directory+'/results.csv')
    else:
        df = pd.DataFrame(columns=['timestamp', 'img_path'])

    # Open detection graph
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
            while True:
                try:
                    # Read frame from video
                    ret, image_np = cap.read()

                    # Copy image for later
                    image_show = np.copy(image_np)
#                     ToWriteOnlabel = ""
                    gunCount=0
                    uniformCount = 0
                    image_height, image_width, _ = image_np.shape
                    # Actual detection.
                    output_dict = run_inference_for_single_image(image_np, sess, tensor_dict)

                    

                    # Get data(label, xmin, ymin, xmax, ymax)
                    output = []
                    for index, score in enumerate(output_dict['detection_scores']):
                            if score < threshold:
                                continue
                            label = category_index[output_dict['detection_classes'][index]]['name']
                            ymin, xmin, ymax, xmax = output_dict['detection_boxes'][index]
                            output.append((label, int(xmin * image_width), int(ymin * image_height), int(xmax * image_width), int(ymax * image_height)))

                    # Save incident (could be extended to send a email or something)
                    for l, x_min, y_min, x_max, y_max in output:

                        if l == label_to_look_for:
                            #CODE FOR SAVING IMAGES. 
#                             array = cv2.cvtColor(np.array(image_show), cv2.COLOR_RGB2BGR)
#                             image = Image.fromarray(array)
#                             cropped_img = image.crop((x_min, y_min, x_max, y_max))
#                             file_path = output_directory+'/images/'+str(len(df))+'.jpg'
#                             cropped_img.save(file_path, "JPEG", icc_profile=cropped_img.info.get('icc_profile'))
#                             df.loc[len(df)] = [datetime.datetime.now(), file_path]
#                             df.to_csv(output_directory+'/results.csv', index=None)
                            #ToWriteOnlabel = "Gun"
                            gunCount=gunCount+1
    
    
                    for l1, x_min1, y_min1, x_max1, y_max in output:
        
                        if l1 == label_to_look_for2:
                            #ToWriteOnlabel = "Uniform"
                            uniformCount = uniformCount+1
                                          
  
                    if show_video_steam:
                        # Visualize results
                        vis_util.visualize_boxes_and_labels_on_image_array(
                            image_np,
                            output_dict['detection_boxes'],
                            output_dict['detection_classes'],
                            output_dict['detection_scores'],
                            category_index,
                            instance_masks=output_dict.get('detection_masks'),
                            use_normalized_coordinates=True,
                            line_thickness=8)
                        if gunCount > uniformCount:
                            showcrime=showcrime+1
                            print(showcrime)
                            
#                             if uniformCount > 2:
#                                 ToWriteOnlabel="POLICE ARRIVED"
#                                 showcrime = 0                            
                            
                            if showcrime > 60:
                                ToWriteOnlabel="CRIME DETECTED"
                                showuniform = 50
                                #showuniform = 5 ##########################new addition########################

                            else:
                                ToWriteOnlabel="NOT A CRIME SCENE"
                            
                            
                        elif uniformCount > gunCount:
                            showuniform=showuniform+1
                            print("showuniform= "+str(showuniform))
                            if showuniform > 70:
                                showcrime = 70
                                ToWriteOnlabel = "NOT A CRIME SCENE [POLICE/ARMY]"
                                
                        
                        if ToWriteOnlabel == "CRIME DETECTED":
                            textColor = (0,0,255)
                        else:
                            textColor = (0,255,0)
                        
                        
                        font         = cv2.FONT_HERSHEY_SIMPLEX
                        bottomLeftCornerOfText = (150,30)
                        fontScale              = 1
                        fontColor              = textColor
                        lineType               = 2

                        cv2.putText(image_np,ToWriteOnlabel, 
                            bottomLeftCornerOfText, 
                            font, 
                            fontScale,
                            fontColor,
                            lineType) 
                        #=================TO DISPLAY GUN COUNT==============================
#                         font         = cv2.FONT_HERSHEY_SIMPLEX
#                         bottomLeftCornerOfText = (500,20)
#                         fontScale              = 1
#                         fontColor              = (255,255,255)
#                         lineType               = 2

#                         cv2.putText(image_np,"gun count = "+str(gunCount), 
#                             bottomLeftCornerOfText, 
#                             font, 
#                             fontScale,
#                             fontColor,
#                             lineType)   
#                         #==================TO DISPLAY UNIFORM COUNT===========================
#                         font         = cv2.FONT_HERSHEY_SIMPLEX
#                         bottomLeftCornerOfText = (500,50)
#                         fontScale              = 1
#                         fontColor              = (255,255,255)
#                         lineType               = 2

#                         cv2.putText(image_np,"uniform count = "+str(uniformCount), 
#                             bottomLeftCornerOfText, 
#                             font, 
#                             fontScale,
#                             fontColor,
#                             lineType)                         
                        
                        cv2.imshow('object_detection', cv2.resize(image_np, (800, 600)))
                        if cv2.waitKey(25) & 0xFF == ord('q'):
                            cap.release()
                            cv2.destroyAllWindows()
                            break

                except Exception as e:
                    print(e)
                    cap.release()
                    cv2.destroyAllWindows()
                    break


# if __name__ == '__main__':
#     parser = argparse.ArgumentParser(description='Surveillance System')
#     parser.add_argument('-m', '--model_path', type=str, required=True, help='Path to the frozen inference graph')
#     parser.add_argument('-l', '--labelmap', type=str, required=True, help='Path to labelmap')
#     parser.add_argument('-t', '--threshold', type=float, default=0.5, help='Threshold for bounding boxes')
#     parser.add_argument('-s', '--show', default=True, action='store_true', help='Show window')
#     parser.add_argument('-la', '--label', default='gun', type=str, help='Label name to detect')
#     parser.add_argument('-o', '--output_directory', default='results', type=str, help='Directory for the outputs')
#     args = parser.parse_args()
#     model1/frozen_inference_graphFinal/frozen_inference_graph_enhanced
run_surveillance("C:/Users/AALY/object_detection/frozen_inference_graph_10-jun-2020.pb", 
                 "C:/Users/AALY/object_detection/label_mapFinal.pbtxt", True, 
                 "gun", "C:/Users/AALY/results", 0.5, "uniform")

1
2
3
4
5
6
7
8
9
10
11
12
showuniform= 1
showuniform= 2
showuniform= 3
showuniform= 4
showuniform= 5
showuniform= 6
showuniform= 7
showuniform= 8
showuniform= 9
showuniform= 10
showuniform= 11
showuniform= 12
showuniform= 13
showuniform= 14
showuniform= 15
showuniform= 16
showuniform= 17
showuniform= 18
showuniform= 19
showuniform= 20
showuniform= 21
showuniform= 22
showuniform= 23
showuniform= 24
showuniform= 25
showuniform= 26
showuniform= 27
showuniform= 28
showuniform= 29
showuniform= 30
showuniform= 31
showuniform= 32
showuniform= 33
showuniform= 34
showuniform= 35
showuniform= 36
showuniform= 37
showuniform= 38
showuniform= 39
showuniform= 40
showuniform= 41
showuniform= 42
showuniform= 43
showuniform= 44
showuniform= 45
showuniform= 46
showuniform= 47
showuniform= 48
showuniform= 49
showuniform= 50
showuniform= 51
showuniform= 52
showuniform= 53
showuniform= 54
showuniform= 55
showuniform= 56
showuniform= 57
showuniform= 58
showuniform= 59
showuniform= 60
showuniform= 61
showun

In [2]:
print("NOT A CRIME SCENE [POLICE/ARMY]")

NOT A CRIME SCENE [POLICE/ARMY]
